# Memory

> Memory modules for benchmarking

In [ ]:
#| default_exp memory

In [ ]:
#| export
from __future__ import annotations

import warnings
from dataclasses import dataclass, asdict
from typing import Dict, Iterable, List
from fasterbench.core import _bytes_to_mib

import numpy as np
import torch

try:
    import psutil
except ImportError:
    psutil = None

In [ ]:
#| export
@dataclass(slots=True)
class MemoryMetrics:
    """Average & peak resident memory (MiB) for one device."""

    avg_mib: float
    peak_mib: float
    reserved_mib: float  # GPU‑only (NaN for CPU)
    device: str

    def as_dict(self) -> Dict[str, float | str]:
        return asdict(self)


#| export
def _gpu_metrics(
    model: torch.nn.Module,
    dummy_input: torch.Tensor,
    *,
    warmup_rounds: int,
    test_rounds: int,
) -> MemoryMetrics:
    dev = torch.device("cuda")
    model = model.eval().to(dev)
    dummy_input = dummy_input.to(dev)

    # warm‑up
    for _ in range(warmup_rounds):
        model(dummy_input)
    torch.cuda.synchronize(dev)

    alloc, alloc_peak, reserv_peak = [], [], []
    for _ in range(test_rounds):
        torch.cuda.reset_peak_memory_stats(dev)
        model(dummy_input)
        torch.cuda.synchronize(dev)
        alloc.append(torch.cuda.memory_allocated(dev))
        alloc_peak.append(torch.cuda.max_memory_allocated(dev))
        reserv_peak.append(torch.cuda.max_memory_reserved(dev))

    return MemoryMetrics(
        avg_mib=_bytes_to_mib(float(np.mean(alloc))),
        peak_mib=_bytes_to_mib(float(np.mean(alloc_peak))),
        reserved_mib=_bytes_to_mib(float(np.mean(reserv_peak))),
        device="cuda",
    )

#| export
def _cpu_metrics(
    model: torch.nn.Module,
    dummy_input: torch.Tensor,
    *,
    warmup_rounds: int,
    test_rounds: int,
) -> MemoryMetrics:
    if psutil is None:
        warnings.warn("psutil not available – returning NaNs for CPU memory metrics")
        nan = float("nan")
        return MemoryMetrics(nan, nan, nan, "cpu")

    proc = psutil.Process()
    model = model.eval().cpu()
    dummy_input = dummy_input.cpu()

    # establish baseline RSS before any warm‑up
    rss0 = proc.memory_info().rss

    for _ in range(warmup_rounds):
        model(dummy_input)

    diffs: List[int] = []
    peaks: List[int] = []
    for _ in range(test_rounds):
        rss_before = proc.memory_info().rss
        model(dummy_input)
        rss_after = proc.memory_info().rss
        diffs.append(max(0, rss_after - rss_before))
        peaks.append(max(0, rss_after - rss0))

    return MemoryMetrics(
        avg_mib=_bytes_to_mib(float(np.mean(diffs))),
        peak_mib=_bytes_to_mib(float(np.max(peaks))),
        reserved_mib=float("nan"),
        device="cpu",
    )


#| export
def compute_memory(
    model: torch.nn.Module,
    dummy_input: torch.Tensor,
    *,
    warmup_rounds: int = 10,
    test_rounds: int = 100,
) -> MemoryMetrics:
    """Alias for GPU if available, else CPU."""
    if torch.cuda.is_available():
        return _gpu_metrics(model, dummy_input, warmup_rounds=warmup_rounds, test_rounds=test_rounds)
    return _cpu_metrics(model, dummy_input, warmup_rounds=warmup_rounds, test_rounds=test_rounds)

#| export
def compute_memory_multi(
    model: torch.nn.Module,
    dummy_input: torch.Tensor,
    *,
    devices: Iterable[str] | None = None,
    warmup_rounds: int = 10,
    test_rounds: int = 100,
) -> Dict[str, MemoryMetrics]:
    """Return memory metrics for each device in *devices* (default cpu + cuda)."""

    if devices is None:
        devices = ["cpu"]
        if torch.cuda.is_available():
            devices.append("cuda")

    metrics: Dict[str, MemoryMetrics] = {}
    for d in devices:
        if d == "cuda" and not torch.cuda.is_available():
            continue
        if d == "cuda":
            metrics[d] = _gpu_metrics(
                model, dummy_input, warmup_rounds=warmup_rounds, test_rounds=test_rounds
            )
        else:
            metrics[d] = _cpu_metrics(
                model, dummy_input, warmup_rounds=warmup_rounds, test_rounds=test_rounds
            )
    return metrics